In [96]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import dashboard components
import dash_leaflet as dl
from dash import dcc, html
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd

# FIX ME: change AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

# Infer Jupyter proxy
JupyterDash.infer_jupyter_proxy_config()

###########################
# Data Manipulation / Model
###########################
# FIX ME: update with your username and password
username = "aacuser"
password = "AACUSER_PASS"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME: Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Dashboard title
    html.Center(html.B(html.H1('CS-340 Dashboard - Nicholas Bean'))),
    
    # Dashboard logo
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                         id='grazioso-logo-nb-2025',
                         style={'width': '300px', 'height': '300px'})),
    html.Hr(),
    
    # Filter dropdown
    # FIXME: Add in code for the interactive filtering options
    html.Div([
        dcc.Dropdown(
            id='filter-type',
            options=[
                {"label": "All Animals", "value": "all"},
                {"label": "Water Rescue", "value": "water"},
                {"label": "Mountain/Wilderness Rescue", "value": "mountain"},
                {"label": "Disaster/Tracking", "value": "disaster"},
            ],
            value="all",
            clearable=False,
            placeholder="Select Rescue Type"
        )
    ]),
    html.Hr(),
    
    # Data table
    # FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native"
    ),
    html.Br(),
    html.Hr(),
    
    # Graph + Map side by side
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', style={'flex': '0 0 65%', 'margin-right': '20px'}),
        html.Div(id='map-id', style={'flex': '0 0 35%'})
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filter data table based on dropdown
@app.callback(
    Output('datatable-id','data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    ## FIX ME: Add code to filter interactive data table with MongoDB queries
    # Water Rescue filter
    if filter_type == "water":
        query = {
            "$and": [
                {"breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}},
                {"sex_upon_outcome": "Intact Female"},
                {"age_upon_outcome_in_weeks": {"$gte":26,"$lte":156}}
            ]
        }
    # Mountain/Wilderness Rescue filter
    elif filter_type == "mountain":
        query = {
            "$and": [
                {"breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte":26,"$lte":156}}
            ]
        }
    # Disaster/Tracking filter
    elif filter_type == "disaster":
        query = {
            "$and": [
                {"breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte":20,"$lte":300}}
            ]
        }
    # Default: show all animals
    else:
        query = {}

    # Run query
    results = db.read(query)
    dff = pd.DataFrame.from_records(results)
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')


# Pie chart of breed distribution
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    if not viewData:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return []

    # Group by breed and sum counts
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']

    # Filter out breeds with very small counts (<1% of total)
    threshold = 0.01 * breed_counts['count'].sum()
    breed_counts = breed_counts[breed_counts['count'] >= threshold]

    import plotly.express as px
    fig = px.pie(
        breed_counts,
        names='breed',
        values='count',
        title='Breed Distribution of Selected Animals',
        hover_data={'count': True, 'breed': True}
    )
    # Hover Template
    fig.update_traces(hovertemplate='%{label}: %{value} animals<extra></extra>')
    return [dcc.Graph(figure=fig)]


# Highlight selected columns in table
# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [({
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    }) for i in selected_columns]


# Map callback with working popups
# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows'),
     Input('filter-type', 'value')]  # Added filter input to prevent all-animals map
)
def update_map(viewData, selected_rows, filter_type):
    # Return empty map if no data or "All Animals" selected
    if not viewData or filter_type == "all":
        return [
            dl.Map(style={'width': '100%', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10,
                   children=[dl.TileLayer()])
        ]

    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [
            dl.Map(style={'width': '100%', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10,
                   children=[dl.TileLayer()])
        ]

    # Build markers for filtered rows only
    markers = []
    for i, row in dff.iterrows():
        lat = row['location_lat']
        lon = row['location_long']
        breed = row['breed']
        name = row['name']

        # Invalid coordinates
        if pd.isna(lat) or pd.isna(lon):
            continue

        # Add marker
        markers.append(
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup(html.Div([
                    html.H4("Animal Name"),
                    html.P(name)
                ]))
            ])
        )

    # Center map on first valid marker
    center = [30.75, -97.48]
    valid_coords = dff[['location_lat', 'location_long']].dropna()
    if not valid_coords.empty:
        center = [valid_coords.iloc[0]['location_lat'], valid_coords.iloc[0]['location_long']]

    return [
        dl.Map(style={'width': '100%', 'height': '500px'},
               center=center, zoom=10,
               children=[dl.TileLayer()] + markers)
    ]


# Run the app
# Run app and display result in jupyterlab mode
app.run_server()

Dash app running on https://dependrelax-barcodejester-3000.codio.io/proxy/8050/
